In [97]:
import pandas as pd
import numpy as np
import torch 

In [98]:
df=pd.read_excel(r'C:\Users\seven\Downloads\google.xlsx')

In [99]:
df2=df['Open']
df2

0        211.47
1        222.17
2        223.22
3        228.66
4        233.44
         ...   
3014    1070.00
3015    1068.64
3016    1066.60
3017    1062.25
3018    1055.49
Name: Open, Length: 3019, dtype: float64

In [100]:
A1=df2.to_numpy()
A1.shape

(3019,)

In [101]:
row_number=3019 - 6 + 1
row_number

3014

In [102]:
idx=(np.tile(np.arange(0,6),(row_number,1)).T + np.arange(0,row_number)).T
A = A1[idx]
A.shape

(3014, 6)

In [103]:
X = A[:,0:5]
Y = A[:,-1]
X.shape

(3014, 5)

In [104]:
X = X.astype(np.float32)
Y = Y.astype(np.float32)

In [105]:
from torch.utils.data import DataLoader

class my_dataset(torch.utils.data.Dataset):
    def __init__(self, ):
        pass

    def __getitem__(self, index):
        return X[index],Y[index]

    def __len__(self):
        return len(X)

ds=my_dataset()

In [106]:
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data import DataLoader
# 划分训练和测试的数据
train, test = random_split(ds, (2800, 214) )
# 设定每次读取的数量
data_loader = torch.utils.data.DataLoader(dataset=train,batch_size=200, shuffle=True)
data_loader2 = torch.utils.data.DataLoader(dataset=test,batch_size=200, shuffle=True)

In [107]:
import torch.nn as nn

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.L1 = nn.Linear(5,100)
        # self.B1= nn.BatchNorm2d(100)
        self.L2 = nn.Linear(100,200)
        # self.B2= nn.BatchNorm2d(200)
        self.L = nn.Linear(200,1)

    def forward(self, x):
        #降二维，从start_dim开始相乘
        # x = torch.flatten(x,start_dim = 1)
        g1 = self.L1(x)
        # g1 = self.B1(g1)
        # f1=torch.sigmoid(g1)
        f1=torch.relu(g1)

        g2 = self.L2(f1)
        # g2 = self.B2(g2)
        f2 = torch.relu(g2)

        g = self.L(f2)

        return g
    
model = MyNet()

In [108]:
import torchmetrics

lossf = nn.MSELoss()     #loss函数
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  #SGD函数
metricsf = torchmetrics.MeanSquaredError()       #torchmetrics函数

for i in range(100):
    for batchX, batchY in data_loader:
        score = model(batchX)
        score = torch.squeeze(score)
        loss = lossf(score, batchY)

        loss.backward() #求导
        optimizer.step() #梯度下降
        optimizer.zero_grad() #梯度清零

        mse = metricsf(score, batchY)    
    print(loss, metricsf.compute() ** 0.5)
    metricsf.reset()

tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, grad_fn=<MseLossBackward0>) tensor(nan)
tensor(nan, 

In [109]:
torch.save(model, 'model6.pth')

In [110]:
for batchX, batchY in data_loader2:
    score = model(batchX)
    score = torch.squeeze(score)
    mse = metricsf(score, batchY) 
      
print(metricsf.compute() ** 0.5 )

tensor(nan)
